In [38]:
import numpy as np
import pandas as pd
import os
train_Y = []
def dataframe_train(path):
    df = pd.read_csv(path,header=0,parse_dates=[1,4,6])
    train_Y = df['Fare']
    return df,train_Y

def dataframe_test(path):
    df = pd.read_csv(path,header=0,parse_dates=[1,4,6])
    return df

In [39]:
df_train,trainY = dataframe_train('/home/arkadeep/Desktop/JP Morgan/train.csv')
df_test = dataframe_test('/home/arkadeep/Desktop/JP Morgan/test.csv')
df_train['diff'] = df_train['Flight Date'] - df_train['Booking Date']
df_test['diff'] = df_test['Flight Date'] - df_test['Booking Date']
df_train['diff'] = df_train['diff'] / np.timedelta64(1, 'D')
df_test['diff'] = df_test['diff']/np.timedelta64(1,'D')
df_train['age'] = df_train['Booking Date'] - df_train['Date of Birth']
df_train['age'] = df_train['age'] / np.timedelta64(1, 'D')
df_test['age'] = df_test['Booking Date'] - df_test['Date of Birth']
df_test['age'] = df_test['age'] / np.timedelta64(1, 'D')
df_train['flight_month'] = df_train['Flight Date'].dt.month
df_test['flight_month'] = df_test['Flight Date'].dt.month
df_train['WEEKEND'] = ((pd.DatetimeIndex(df_train['Flight Date']).dayofweek)//5+1).astype(float)
df_test['WEEKEND'] = ((pd.DatetimeIndex(df_test['Flight Date']).dayofweek)//5+1).astype(float)
df_train['FLIGHTWEEK'] = df_train['Flight Date'].dt.week
df_test['FLIGHTWEEK'] = df_test['Flight Date'].dt.week
#df_test['WEEKDAY'] = (pd.DatetimeIndex(df_test['Booking Date']).dayofweek).astype(float)
#df_train['WEEKDAY'] = (pd.DatetimeIndex(df_test['Booking Date']).dayofweek).astype(float)
df_train['WEEKDAY'] = df_train['Flight Date'].dt.dayofweek
df_test['WEEKDAY'] = df_train['Flight Date'].dt.dayofweek

In [40]:
holidays = ['2016-01-01','2016-01-14','2016-01-15','2016-01-16','2016-01-26','2016-02-12','2016-03-07','2016-03-23','2016-03-25','2016-03-27','2016-04-13','2016-04-15','2016-04-20','2016-04-21','2016-05-21','2016-07-08','2016-08-15','2016-08-18','2016-08-25','2016-09-05','2016-09-12','2016-09-14','2016-10-02','2016-10-11','2016-10-12','2016-10-30','2016-10-31','2016-11-01','2016-11-06','2016-11-24','2016-12-13','2016-12-24','2016-12-25']

In [41]:
imp = {'Mumbai':1 , 'Patna' : 2, 'Kolkata' : 3, 'Delhi' : 4, 'Hyderabad': 5, 'Lucknow': 6, 'Chennai':7}

In [42]:
dict_cities = {'Mumbai':1 , 'Patna' : 2, 'Kolkata' : 3, 'Delhi' : 4, 'Hyderabad': 5, 'Lucknow': 6, 'Chennai':7}
dict_class = {'Business':25, 'Economy': 10}
df_train = df_train.replace({'From':dict_cities})
df_test = df_test.replace({'From':dict_cities})
df_train = df_train.replace({'To': dict_cities})
df_test = df_test.replace({'To': dict_cities})
df_train = df_train.replace({'Class':dict_class})
df_test = df_test.replace({'Class':dict_class})

In [43]:
def label_holiday (row):
    if str(row['Flight Date']) in holidays :
        return 3
    elif row['Flight Date'].dayofweek //5 ==1:
        return 2
    else:
        return 1
    return None
df_train['WEEKEND'] = df_train.apply (lambda row: label_holiday(row),axis=1)
df_test['WEEKEND'] = df_test.apply (lambda row: label_holiday(row),axis=1)

In [44]:
df_train = df_train.drop(columns=['Flight Date', 'Booking Date'])
df_test = df_test.drop(columns=['Flight Date', 'Booking Date'])
df_train = df_train.drop(columns='Date of Birth')
df_test = df_test.drop(columns='Date of Birth')

In [45]:
df_train['Flight Time'] = df_train['Flight Time'].str.split(':').str[0]
df_train['age'] = df_train['age']/365.0
df_test['Flight Time'] = df_test['Flight Time'].str.split(':').str[0]
df_test['age'] = df_test['age']/365.0

In [46]:
time_from = {}
time_from['1'] = [0,130,140,120,75,130,89]
time_from['2'] = [205,0,60,95,104,39,134]
time_from['3'] = [155,90,0,120,120,105,135]
time_from['4'] = [115,80,115,0,120,55,155]
time_from['5'] = [70,104,115,125,0,235,65]
time_from['6'] = [130,39,100,60,245,0,138]
time_from['7'] = [95,134,115,155,65,138,0]

In [47]:
def label_duration (row):
    return time_from[str(int(row['From']))][int(row['To'])-1]

def label_distance (row):
   if (row['From'] == 1 and row['To'] == 2) or (row['From'] == 2 and row['To'] == 1):
        return 1446
   if (row['From'] == 1 and row['To'] == 3) or (row['From'] == 3 and row['To'] == 1):
        return 1654
   if (row['From'] == 1 and row['To'] == 4) or (row['From'] == 4 and row['To'] == 1):
        return 1148
   if (row['From'] == 1 and row['To'] == 5) or (row['From'] == 5 and row['To'] == 1):
        return 622
   if (row['From'] == 1 and row['To'] == 6) or (row['From'] == 6 and row['To'] == 1):
        return 1190
   if (row['From'] == 1 and row['To'] == 7) or (row['From'] == 7 and row['To'] == 1):
        return 1028
   if (row['From'] == 2 and row['To'] == 3) or (row['From'] == 3 and row['To'] == 2):
        return 470
   if (row['From'] == 2 and row['To'] == 4) or (row['From'] == 4 and row['To'] == 2):
        return 480
   if (row['From'] == 2 and row['To'] == 5) or (row['From'] == 5 and row['To'] == 2):
        return 1140
   if (row['From'] == 2 and row['To'] == 6) or (row['From'] == 6 and row['To'] == 2):
        return 437
   if (row['From'] == 2 and row['To'] == 7) or (row['From'] == 7 and row['To'] == 2):
        return 1485
   if (row['From'] == 3 and row['To'] == 4) or (row['From'] == 4 and row['To'] == 3):
        return 1307
   if (row['From'] == 3 and row['To'] == 5) or (row['From'] == 5 and row['To'] == 3):
        return 1180
   if (row['From'] == 3 and row['To'] == 6) or (row['From'] == 6 and row['To'] == 3):
        return 886
   if (row['From'] == 3 and row['To'] == 7) or (row['From'] == 7 and row['To'] == 2):
        return 1366
   if (row['From'] == 4 and row['To'] == 5) or (row['From'] == 5 and row['To'] == 4):
        return 1253
   if (row['From'] == 4 and row['To'] == 6) or (row['From'] == 6 and row['To'] ==  4):
        return 417
   if (row['From'] == 4 and row['To'] == 7) or (row['From'] == 7 and row['To'] == 4):
        return 1760
   if (row['From'] == 5 and row['To'] == 6) or (row['From'] == 6 and row['To'] == 5):
        return 1425
   if (row['From'] == 5 and row['To'] == 7) or (row['From'] == 7 and row['To'] == 5):
        return 520
   if (row['From'] == 6 and row['To'] == 7) or (row['From'] == 7 and row['To'] == 6):
        return 1534
   return None

In [48]:
df_train['flight_duration'] = df_train.apply (lambda row: label_duration (row),axis=1)
df_test['flight_duration'] = df_test.apply (lambda row: label_duration (row),axis=1)
df_train['distance'] = df_train.apply (lambda row: label_distance (row),axis=1)
df_test['distance'] = df_test.apply (lambda row: label_distance (row),axis=1)

In [49]:
df_train['Flight Time'] = df_train['Flight Time'].astype(str).astype(int)
df_test['Flight Time'] = df_test['Flight Time'].astype(str).astype(int)

In [50]:
df_train['Name'] = df_train['Name'].str.split('.').str[0]
df_test['Name'] = df_test['Name'].str.split('.').str[0]

In [51]:
def label_gender (row):
    if 'Dr' in row['Name']:
        return 4
    if 'Miss' in row['Name']:
        return 0
    if 'Mrs' in row['Name']:
        return 1
    if 'Mr' in row['Name']:
        return 3
    return None

In [52]:
df_train['gender'] = df_train.apply (lambda row: label_gender (row),axis=1)
df_test['gender'] = df_test.apply (lambda row: label_gender (row),axis=1)
df_train= df_train.drop(columns='Name')
df_test = df_test.drop(columns='Name')
df_train =df_train.drop(columns='Fare')
df_train

,From,To,Flight Time,Class,diff,age,flight_month,WEEKEND,FLIGHTWEEK,WEEKDAY,flight_duration,distance,gender
0,1,3,13,25,16.0,53.027397,11,1,44,4,140,1654.0,4
1,6,5,18,10,50.0,34.832877,9,1,38,0,245,1425.0,4
2,6,2,10,10,3.0,34.898630,5,1,21,4,39,437.0,4
3,1,6,14,10,8.0,27.871233,7,2,29,6,130,1190.0,3
4,3,4,10,10,12.0,22.073973,12,1,51,2,120,1307.0,0
5,7,2,14,25,10.0,10.501370,5,1,20,1,134,1485.0,0
6,1,6,11,25,5.0,67.734247,5,1,21,2,130,1190.0,3
7,7,1,18,10,24.0,72.821918,5,1,20,4,95,1028.0,4
8,4,6,12,10,0.0,44.567123,11,2,47,6,55,417.0,3
9,4,1,12,10,1.0,66.542466,4,2,14,6,115,1148.0,3


In [34]:
imp = {1:6 , 2 : 1, 3 : 3, 4 : 7, 5: 4, 6: 2, 7:5}
def label_imp (row):
    return int(imp[int(row['From'])])
    return None
df_train['priority'] = df_train.apply(lambda row: label_imp(row),axis=1)
df_test['priority'] = df_test.apply(lambda row: label_imp(row),axis=1)
df_train['priority']

0       6
1       2
2       2
3       6
4       3
5       5
6       6
7       5
8       7
9       7
10      1
11      7
12      5
13      3
14      2
15      6
16      6
17      3
18      6
19      7
20      6
21      2
22      2
23      5
24      7
25      7
26      3
27      6
28      2
29      3
       ..
7470    3
7471    4
7472    7
7473    7
7474    3
7475    3
7476    6
7477    2
7478    5
7479    7
7480    2
7481    3
7482    7
7483    6
7484    1
7485    4
7486    2
7487    1
7488    3
7489    1
7490    1
7491    6
7492    6
7493    7
7494    4
7495    2
7496    6
7497    3
7498    2
7499    7
Name: priority, Length: 7500, dtype: int64

In [35]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
xgb = XGBRegressor(max_depth = 20,objective='reg:linear',n_estimators=500)
xgb.fit(df_train,trainY)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=20, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [36]:
xgb.score(df_train,trainY)

0.9999999999996952

In [37]:
xgb_final_preds = xgb.predict(df_test)
np.savetxt('finall_lasttttttttttt_946.csv',xgb_final_preds,delimiter = ',')